# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [4]:
import pandas as pd
import platform

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from random import randint

In [5]:
# На машинах исполнителя проекта файл с данными доступен через символический путь, 
# ссылающийса на папку на яндекс.диске, где расположен загружаемый файл. 
# В случае если хостом где выполняется анализ является какая-либо другая машина, 
# используем путь по умолчанию.
try:    
    host = platform.node()
    if host in ['22varivoda','Gover-pc','MSI']:
        filepath    = 'C:/_YDsymlink/Python/datascience/Projects/10 - Введение в машинное обучение/users_behavior.csv'
    else:
        filepath    = '/datasets/users_behavior.csv'
except:
    print("Не удалось подгрузить данные")

In [6]:
df = pd.read_csv(filepath)

Посмотрим на образец исходных данных

In [7]:
df.sample(10)

,calls,minutes,messages,mb_used,is_ultra
1698,26.0,149.13,23.0,17022.64,0
264,0.0,0.00,21.0,19559.55,0
841,59.0,371.10,0.0,14647.95,0
1281,148.0,1115.64,41.0,25465.40,1
3155,64.0,431.79,14.0,14675.42,0
3149,78.0,503.55,57.0,15258.51,0
1790,32.0,220.55,140.0,10349.31,1
379,51.0,426.24,112.0,5661.23,0
2275,58.0,395.81,77.0,25859.09,0
272,69.0,564.01,26.0,15656.78,0


## Разбейте данные на выборки

Фиксированный гиперпараметр: random_state = 777

In [8]:
const_random_state = 777

Поделим датасет на три выборки: обучающая 60%, валидационная 20%, тестовая 20% (пропорция 3/1/1).

In [9]:
df_train, df_valid_and_test = train_test_split(df, train_size=0.60, random_state = const_random_state)
df_valid, df_test = train_test_split(df_valid_and_test, train_size = 0.5)

# Разбиваем выборки на признаки
features_train = df_train.drop(columns='is_ultra')
target_train   = df_train['is_ultra']

features_valid = df_valid.drop(columns='is_ultra')
target_valid   = df_valid['is_ultra']

features_test  = df_test.drop(columns='is_ultra')
target_test    = df_test['is_ultra']

Проверка на результат разбиения:

In [12]:
print(f"features_train содержит записи для {len(features_train)} объектов, что составляет {len(features_train)/len(df):.2%} от изначального количества")
print(f"target_train содержит записи для {len(target_train)} объектов, что составляет {len(target_train)/len(df):.2%} от изначального количества")
print(f"features_valid содержит записи для {len(features_valid)} объектов, что составляет {len(features_valid)/len(df):.2%} от изначального количества")
print(f"target_valid содержит записи для {len(target_valid)} объектов, что составляет {len(target_valid)/len(df):.2%} от изначального количества")
print(f"features_test содержит записи для {len(features_test)} объектов, что составляет {len(features_test)/len(df):.2%} от изначального количества")
print(f"target_test содержит записи для {len(target_test)} объектов, что составляет {len(target_test)/len(df):.2%} от изначального количества")

features_train содержит записи для 1928 объектов, что составляет 59.99% от изначального количества
target_train содержит записи для 1928 объектов, что составляет 59.99% от изначального количества
features_valid содержит записи для 643 объектов, что составляет 20.01% от изначального количества
target_valid содержит записи для 643 объектов, что составляет 20.01% от изначального количества
features_test содержит записи для 643 объектов, что составляет 20.01% от изначального количества
target_test содержит записи для 643 объектов, что составляет 20.01% от изначального количества


## Исследуйте модели

Будем пробовать разные модели обучения в следующем порядке: дерево, лес, логистическая регрессия.<BR>
Для каждой модели будем менять в цикле гиперпараметры и считать для них точность предсказаний. Наиболее удачные наборы гипераметров сохраним в отдельные переменные.

### Дерево

Диапазоны гиперпараметров:<BR>
- max_depth - от 1 до 10
- min_samples_split от 2 до 6
- min_samples_split от 1 до 5

In [ ]:
tree_best_depth = 0
tree_best_leaf_samples = 0
tree_best_split_samples = 0
tree_best_accuracy = 0
accuracy = 0

In [ ]:
for depth in range(1,11):
    for split_samples in range(2,7):
        for leaf_samples in range(1,6):
            # Инициализируем модель            
            model = DecisionTreeClassifier(
                max_depth = depth,
                min_samples_split = split_samples,
                min_samples_leaf  = leaf_samples,
                random_state = const_random_state
            )
            
            # Обучаем
            model.fit(features_train, target_train)
            
            # Проверяем на валидационной выборке
            predictions_tree_valid = model.predict(features_valid)
            
            # Проводим оценку точности
            accuracy = accuracy_score(target_valid, predictions_tree_valid)
            
            if accuracy > tree_best_accuracy:
                tree_best_accuracy = accuracy
                tree_best_depth = depth
                tree_best_split_samples = split_samples                
                tree_best_leaf_samples  = leaf_samples 
                tree_best_model = model

print(f'Алгорим "Дерево" (при random_state={const_random_state}):')
print(f'   Наилучшая максимальная глубина: {tree_best_depth}')
print(f'   Наилучшее значение min_samples_split: {tree_best_split_samples}')
print(f'   Наилучшая значение min_samples_leaf: {tree_best_leaf_samples}')
print(f'   Точность: {tree_best_accuracy:.2f}')

### Лес

Диапазоны гиперпараметров:

- max_depth - от 1 до 10
- n_estimators - от 10 до 200 с шагом 10
- min_samples_split от 2 до 6
- min_samples_split от 1 до 5

In [ ]:
forest_best_depth = 0
forest_best_n_estimators = 0
forest_best_leaf_samples = 0
forest_best_split_samples = 0
forest_best_accuracy = 0
accuracy = 0

Запускаем длительный процесс выбора гиперпараметров (около 10-15 минут). Жертвуем временем на поиск эффективных параметров обучения для получения наилучшего результата

Закомментируем оригинальные строки с перебором гиперпараметров чтобы не тратить много времени  на отработку циклов. Оставляю "иммитацию" с тем параметром который в итоге получился, чтобы всё отработало быстро.

In [ ]:
# ИММИТАЦИЯ
for depth in range(10,11):
    for n_est in range(50,51,10):
        for split_samples in range(2,3):
            for leaf_samples in range(3,4):
                
                # Инициализируем модель            
                model = RandomForestClassifier(
                            max_depth = depth,
                            n_estimators = n_est,
                            min_samples_split = split_samples,
                            min_samples_leaf = leaf_samples,
                            random_state = const_random_state
                        )
            
                # Обучаем модель
                model.fit(features_train, target_train)
            
                # Предсказываем по валидационному набору данных
                predictions_forest_valid = model.predict(features_valid)
            
                # Проводим оценку
                accuracy = accuracy_score(target_valid, predictions_forest_valid)
            
                if accuracy > forest_best_accuracy:
                    forest_best_accuracy = accuracy
                    forest_best_depth = depth
                    forest_best_n_estimators = n_est
                    forest_best_split_samples = split_samples
                    forest_best_leaf_samples  = leaf_samples            
                    forest_best_model = model

print(f'Алгорим "Лес" (при random_state={const_random_state}):')
print(f'   Наилучшая максимальная глубина: {forest_best_depth}')
print(f'   Наилучшее число деревьев (n_estimators): {forest_best_n_estimators}')
print(f'   Наилучшее значение min_samples_split: {forest_best_split_samples}')
print(f'   Наилучшая значение min_samples_leaf: {forest_best_leaf_samples}')
print(f'   Точность: {forest_best_accuracy:.2f}')

### Логистическая регрессия

Диапазоны гиперпараметров:

- max_iter - от 80 до 200

In [ ]:
log_reg_max_iter = 0
log_reg_best_accuracy = 0
accuracy = 0

In [ ]:
for m_iter in range(80,200):
    # Инициализируем модель
    model = LogisticRegression(
        max_iter = m_iter, 
        solver = 'lbfgs',
        random_state = const_random_state)
    
    # Обучаем модель
    model.fit(features_train,target_train)
    
    # Предсказываем целевой признак
    predictions_log_reg = model.predict(features_valid)
    
    # Проверяем точность предсказаний, сравниваем с точностью при других значениях итераций
    accuracy = model.score(features_valid, target_valid)
    
    if accuracy > log_reg_best_accuracy:
        log_reg_best_accuracy = accuracy
        log_reg_max_iter = m_iter
        log_reg_best_model = model

print(f'Алгорим "Логистическая регрессия" (при random_state={const_random_state}):')
print(f'   Наилучшее количество итераций: {log_reg_max_iter}')
print(f'   Точность: {log_reg_best_accuracy:.2f}') 

## Проверьте модель на тестовой выборке

Ещё раз подводим итоги сравнения моделей:

In [ ]:
print(f'Лучшее значение точности по модели "Дерево решений" {tree_best_accuracy:.2f}')
print(f'Лучшее значение точности по модели "Случайный лес" {forest_best_accuracy:.2f}')
print(f'Лучшее значение точности по модели "Логистическая регрессия" {log_reg_best_accuracy:.2f}')

Делаем проверку с помощью тестовой выборки

In [ ]:
tree_predict_test   = tree_best_model.predict(features_test)
forest_predict_test = forest_best_model.predict(features_test)
log_reg_predict_test= log_reg_best_model.predict(features_test)

tree_test_accuracy_value   = accuracy_score(target_test, tree_predict_test   )
forest_test_accuracy_value = accuracy_score(target_test, forest_predict_test )
log_reg_accuracy_value     = accuracy_score(target_test, log_reg_predict_test)

In [ ]:
print(f'Точность модели "Дерево решений" на тестовой выборке {tree_test_accuracy_value:.2f}')
print(f'Точность модели "Случайный лес" на тестовой выборке {forest_test_accuracy_value:.2f}')
print(f'Точность модели "Логистическая регрессия" на тестовой выборке {log_reg_accuracy_value:.2f}')

<b>Вывод:</b><BR>
Модель "Случайный лес" с гиперпараметрами: <BR>
<code><BR>Максимальная глубина (max_depth): 10
Число деревьев (n_estimators): 50
min_samples_split: 2
min_samples_leaf: 3
</code><BR>
является предпочтительной так как дает наибольшую точность предсказаний (около 79-81%). Эта точность достигнута результатом перебора всех перечисленных гиперпараметров в помощью циклов. Процесс перебора довольно длительный (по сравнению с моделями "дерево решений" и "логистическая регрессия", около 10-15 минут), однако дает б<b><i>о</b></i>льшую точность предсказаний.
На втором месте стоит "решающее дерево". Его точность немного уступает "случайному лесу", но не сильно.

## (бонус) Проверьте модели на адекватность

Создадим модель, дающую на выходе случайные значения 0 и 1. Значения будут генерироваться функцией random_model, принимающей на вход датасет из признаков и возвращающей Series, содержащий число записей, соответствующее числу числу строк в полученном датасете признаков.

In [ ]:
def random_model(features):
    random_set = []
    for i in range(0,len(features)):
        random_set.append(randint(0,1))
    return pd.Series(random_set,index=features.index)

In [ ]:
predictions_random = random_model(features_test)

accuracy_random = accuracy_score(predictions_random,target_test)

Сравнение точностей проверенных (на тестовой выборке) моделей:

In [ ]:
print(f'Модель "Решающее дерево": {tree_test_accuracy_value:.2f}')
print(f'Модель "Случайный лес": {forest_test_accuracy_value:.2f}')
print(f'Модель "Логистическая регрессия": {log_reg_accuracy_value:.2f}')
print(f'"Случайная" модель: {accuracy_random:.2f}')

<b>Вывод</b>

Все проверенные модели существенно превосходят по точности "случайную" модель, показывая неплохие результаты.